In [ ]:
#input must be options chain day1 - end

In [1]:
import wandb
import os
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import random
import hashlib

In [2]:
CSV_PATH     = './data/OptionsEOD.csv/'
PARQUET_PATH = './data/OptionsEOD.parquet'
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = ['DTE','INTRINSIC_VALUE','TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']

In [3]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003130,0.002320,0.000350,-0.004080,0.000000,0.793590,,10.9,0.192
1,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003580,0.002780,0.000530,-0.004420,-0.000420,0.721270,,9.9,0.174
2,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.004040,0.003340,0.000950,-0.004350,-0.000430,0.648640,,8.9,0.156
3,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.010000,-0.004390,0.004150,0.001170,-0.004050,-0.000330,0.577770,0.000000,7.9,0.139
4,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,0.971540,0.016360,...,0.010000,-0.004860,0.005270,0.001370,-0.004140,0.000000,0.507560,0.000000,6.9,0.121


In [4]:
EOD_CSV.columns

Index(['[QUOTE_UNIXTIME]', ' [QUOTE_READTIME]', ' [QUOTE_DATE]',
       ' [QUOTE_TIME_HOURS]', ' [UNDERLYING_LAST]', ' [EXPIRE_DATE]',
       ' [EXPIRE_UNIX]', ' [DTE]', ' [C_DELTA]', ' [C_GAMMA]', ' [C_VEGA]',
       ' [C_THETA]', ' [C_RHO]', ' [C_IV]', ' [C_VOLUME]', ' [C_LAST]',
       ' [C_SIZE]', ' [C_BID]', ' [C_ASK]', ' [STRIKE]', ' [P_BID]',
       ' [P_ASK]', ' [P_SIZE]', ' [P_LAST]', ' [P_DELTA]', ' [P_GAMMA]',
       ' [P_VEGA]', ' [P_THETA]', ' [P_RHO]', ' [P_IV]', ' [P_VOLUME]',
       ' [STRIKE_DISTANCE]', ' [STRIKE_DISTANCE_PCT]'],
      dtype='object')

In [ ]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    scaler = MinMaxScaler()
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)


                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [8]:
#Part II 
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
def strikeZero(df,v,num_rm):
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == v['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    
    # print(max_intrinsic_value)
    # print(min_intrinsic_value)
    df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
            & (  (df['STRIKE'].isin(filtered_arr[:5])  )
               | (df['STRIKE'].isin(filtered_arr[-5:])   )
              ) 
        , ['DTE', 'INTRINSIC_VALUE', 'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME']       
    ] = 0
    
def hash_str(S):
    return hashlib.md5(S.encode('utf-8')).hexdigest() 
    
def TransformDataII():
    
    keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()
    max_option_len = 20
    scaler = MinMaxScaler()
    PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
    #PartitionDate = ['2011-12','2022-05'] # debug
    options_qoute = {}
    for i,partdate in enumerate(PartitionDate) :  
        df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                     , filters=[('PartitionDate', '=', partdate)]
                                    )
        #add col options_id
        df['OPTIONS_ID'] = None
        df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
        df['C_VOLUME'] = df['C_VOLUME'].fillna(0)

        ####################################################
        keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
        #loop each keys
        for j,v in keys.iterrows():
            
            df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
            qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
            #add qoute
            if qoute not in [*options_qoute.keys()]:
                options_qoute[qoute] = {}
                options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                #check diff UNDERLYING_LAST
                if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                    print('[ERROR] : set UNDERLYING_LAST ',qoute )
            #rm index max : max_option_len
            rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
            df = df.drop(rm_strike_index)
            df_filter = df_filter.drop(rm_strike_index)

            # Generate zero strike 
            # Generate a random float between 0.01 and 1
            random_number = random.uniform(0, 1)
            nom_rm_rows = [r  for r in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4] if random_number < r]
            strikeZero(df,v,len(nom_rm_rows) )
            strikeZero(df_filter,v,len(nom_rm_rows) )
            #TOTAL_P_VOLUME,TOTAL_C_VOLUME,OPTIONS_ID
            df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
            , ['TOTAL_VOLUME','OPTIONS_ID']
            ] = [
                df_filter['P_VOLUME'].sum() + df_filter['C_VOLUME'].sum(),
                hash_str(f'{i}{j}')
            ]
    
        #clear expire options_qoute
        for qi in list(options_qoute.keys()):
            if options_qoute[qi]['exp'] < df['QUOTE_DATE'].values[0]:
                options_qoute.pop(qi)
#        return df
    ##################### SAVE DATA #############################

        # scaler.partial_fit
        scaler.partial_fit(df[SCALER_COL])
        print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')
        
        # save
        if os.path.exists(PARQUET_STG_PATH):
          df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
        else:
          df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
            
    joblib.dump(scaler, SCALER_PATH )
        

In [ ]:
TransformDataII()

In [ ]:
++++++ TransformDataII DEBUG ++++++ 

In [ ]:
opt_ids=np.unique( df['OPTIONS_ID'])
for opt_id in opt_ids[:-2]:
    df[  df["OPTIONS_ID"]==opt_id ]

In [ ]:
 df[  df["OPTIONS_ID"]==opt_id ]['P_VOLUME'].sum()

In [ ]:
 df[  df["OPTIONS_ID"]==opt_id ]['C_VOLUME'].sum()

In [ ]:
df[  df["OPTIONS_ID"]==opt_id ][SCALER_COL]

In [ ]:
++++++ TransformDataII DEBUG ++++++ 

In [ ]:
#PartIII to H5

In [ ]:
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
#================== initialization ==================
LATENT_DIM = 128
VAE_LR = 5e-4
PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
H5_PATH = './data/OptTrainData/'
EPOCHS = 5

In [ ]:
#Example
import random
from IPython.display import clear_output,display, HTML
import numpy as np
#load scaler
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
scaler = joblib.load(SCALER_PATH)


for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
    DATA  = np.empty((0,) + (20,9) ) 
    for opt_id in np.unique( df[["OPTIONS_ID"]].values):
        df_filter  = df[df["OPTIONS_ID"]==opt_id]
        if len(df_filter) == 20:
            DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
        else:
            #print( len(df_filter) )
            #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
            pass
            
    # Save the NumPy array to an HDF5 file
    with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
        dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

    print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [ ]:
#ปรับแก้ กรณี rows < 20 and rows > 20
#try save md5

In [ ]:
joblib.dump(scaler, SCALER_PATH )

In [ ]:
#Example data
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_STG_PATH) if 'PartitionDate' in d]
options_qoute = {}
for i,partdate in enumerate(PartitionDate[5:6]) :  
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
        , filters=[('PartitionDate', '=', partdate)]
        )
    break

In [ ]:
keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
#loop each keys
for j,v in keys.iterrows():

    df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
        & (df['SYMBOL'] == v['SYMBOL']) 
        & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
    ]

    df_filter=df_filter[
        (df_filter['INTRINSIC_VALUE'].max() == df_filter['INTRINSIC_VALUE'])
    |   (df_filter['INTRINSIC_VALUE'].min() == df_filter['INTRINSIC_VALUE'])
    ]
    break

In [ ]:
3============== sampling show 3===========================

In [ ]:
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_STG_PATH) if 'PartitionDate' in d]

In [ ]:
import random
from IPython.display import clear_output,display, HTML

for i in range(10):
    ran_partiion=random.choice(PartitionDate)

    df = pd.read_parquet(PARQUET_STG_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', ran_partiion)]
                                )
    key = random.choice( [*df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates().values] )

    df_filtered=df[ (df['QUOTE_DATE'] == key[0]) 
        & (df['SYMBOL'] == key[1]) 
        & (df['EXPIRE_DATE'] == key[2]) 
    ]
    break
    display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
    input('Next ...')
    clear_output()


In [ ]:
=========================================== TEST ==============================================

In [ ]:
keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()
max_option_len = 20
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
options_qoute = {}
for i,partdate in enumerate(PartitionDate) :  
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    #add col options_id
    df['OPTIONS_ID'] = None
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)

    ####################################################
    keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
    #loop each keys
    break_flag = False

    for j,v in keys.iterrows():
        
        df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
        qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
        #add qoute
        if qoute not in [*options_qoute.keys()]:
            options_qoute[qoute] = {}
            options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
            options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
            options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
            #check diff UNDERLYING_LAST
            if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                print('[ERROR] : set UNDERLYING_LAST ',qoute )
        #rm index max : max_option_len
        rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
        if len( df_filter[ df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ] ) != 20 :
            break
        df = df.drop(rm_strike_index)
        df_filter = df_filter.drop(rm_strike_index)

        # Generate zero strike 
        # Generate a random float between 0.01 and 1
        random_number = random.uniform(0, 1)
        nom_rm_rows = [r  for r in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4] if random_number < r]
        strikeZero(df,v,len(nom_rm_rows) )
        #TOTAL_P_VOLUME,TOTAL_C_VOLUME,OPTIONS_ID
        df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
        & (df['SYMBOL'] == v['SYMBOL']) 
        & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
        , ['TOTAL_VOLUME','OPTIONS_ID']
        ] = [
            df_filter['P_VOLUME'].sum() + df_filter['C_VOLUME'].sum(),
            hash_str(f'{i}{j}')
        ]
    if break_flag : 
        break
    #clear expire options_qoute
    for qi in list(options_qoute.keys()):
        if options_qoute[qi]['exp'] < df['QUOTE_DATE'].values[0]:
            options_qoute.pop(qi)

    ###################################################

    # scaler.partial_fit
    #scaler.partial_fit(df[SCALER_COL])
    print(f"[Processing] {partdate} {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')
    
    # save
    # if os.path.exists(PARQUET_STG_PATH):
    #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
    # else:
    #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
        
#joblib.dump(scaler, SCALER_PATH )

In [ ]:
df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]

In [ ]:
df_filter

In [ ]:
type(pd.util.hash_pandas_object(pd.Series([2,1])).sum() )

In [ ]:
pd.util.hash_pandas_object(pd.Series([2,0])).sum()

In [ ]:
pd.util.hash_pandas_object(pd.Series([0,2])).sum()

In [ ]:
import hashlib
a=hashlib.md5("12".encode('utf-8')).hexdigest() 

In [ ]:
def hash_str():
    hashlib.md5("01".encode('utf-8')).hexdigest() 

In [ ]:
a.hexdigest(20)